<a href="https://colab.research.google.com/github/jubaal91/GAAT/blob/main/Introduccion_PR_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Librerías comunes para trabajar con imagenes
import pandas as pd
import geopandas as gpd

#Earth Engine: Nos permite visualizar y manipular imagenes de satélite sin necesidad de descargarlas
import ee

#Permite visualizar imagenes dentro del notebook
from IPython.display import Image

#Leer las fechas
import datetime

In [ ]:
#Autentificación para acceder a EE
ee.Authenticate()

# Inicialización, es necesario definir el proyecto donde se trabajará
ee.Initialize(project='ee-jubaal91')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [ ]:
#Coordenadas del Incendio

lat = 21.998532
lon = -101.040729

#Punto de interés tomando las coordenadas
poi = ee.Geometry.Point(lon,lat)

#Fecha de inicio
start_date = '2024-04-01'

#Fecha final
end_date = '2024-05-31'

In [ ]:
#Obtener las imagenes del satélite Sentinel 2
sentinel = ee.ImageCollection("COPERNICUS/S2_SR")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

In [ ]:
#¿Cuántas imagenes hay de la zona?
print('Total number:', sentinel.size().getInfo())

Total number: 26


In [ ]:
#Información acerca de la primera imagen
#Es util para conocer el nombre de los campos, en caso de que sea necesario consultar información de ellos
sentinel.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32613',
   'crs_transform': [60, 0, 799980, 0, -60, 2500020]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32613',
   'crs_transform': [10, 0, 799980, 0, -10, 2500020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32613',
   'crs_transform': [10, 0, 799980, 0, -10, 2500020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32613',
   'crs_transform': [10, 0, 799980, 0, -10, 2500020]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min':

In [ ]:
#¿Cual es el porcentaje de nubes que cubren a la primera imagen?
sentinel.first().get('CLOUDY_PIXEL_PERCENTAGE').getInfo()

0.003537

In [ ]:
#¿Cuando fue tomada esta imagen?
datetime.datetime.fromtimestamp(sentinel.first().get('system:time_start').getInfo()/1000).strftime('%Y-%m-%d %H:%M:%S')


'2024-04-03 17:27:46'

In [ ]:
#'Cuántas bandas tenemos?
sentinel.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'AOT',
 'WVP',
 'SCL',
 'TCI_R',
 'TCI_G',
 'TCI_B',
 'MSK_CLDPRB',
 'MSK_SNWPRB',
 'MSK_CLASSI_OPAQUE',
 'MSK_CLASSI_CIRRUS',
 'MSK_CLASSI_SNOW_ICE']

In [ ]:
#Guarda todas las imagenes en una lista
sentinel_list = sentinel.toList(sentinel.size());

In [ ]:
#Configura algunos parametros para las imagenes
parameters = {
                'min': 0,
                'max': 3000,
                'dimensions': 800, # square size in pixels
                'bands': ['B4', 'B3', 'B2'] # bands to display (r,g,b)
             }

In [ ]:
import datetime

#Crea un contenedor de datos vacio
data = []

#Ciclo que recorre cada una de las imagenes y las despliega
for i in range(sentinel.size().getInfo()):

    #Fecha de captura de las imagenes
    timestamp_ms = ee.Image(sentinel_list.get(i)).get('system:time_start').getInfo()
    timestamp_s = timestamp_ms / 1000
    date_gregorian = datetime.datetime.fromtimestamp(timestamp_s).strftime('%Y-%m-%d %H:%M:%S')


    #Porcentaje de nubosidad
    cloud = ee.Image(sentinel_list.get(i)).get('CLOUDY_PIXEL_PERCENTAGE').getInfo()

    #Muestra la información de la imagen
    print('Image #',i,date_gregorian,'Cloud cover:',cloud)

    #Despliega la imagen
    display(Image(url = ee.Image(sentinel_list.get(i)).getThumbUrl(parameters)))

    #Convierte esta información (i, fecha, nubosidad) en una lista
    this_data = [i,date_gregorian,cloud]

    #Va agregando información a la lista
    data.append(this_data)


#Crea un data frame
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

Image # 0 2024-04-03 17:27:46 Cloud cover: 0.003537


Image # 1 2024-04-03 17:27:45 Cloud cover: 0.005912


Image # 2 2024-04-08 17:27:48 Cloud cover: 0


Image # 3 2024-04-08 17:27:47 Cloud cover: 0


Image # 4 2024-04-13 17:27:50 Cloud cover: 60.079706


...

Image # 21 2024-05-18 17:27:47 Cloud cover: 0.49068


Image # 22 2024-05-23 17:27:47 Cloud cover: 0


Image # 23 2024-05-23 17:27:46 Cloud cover: 0


Image # 24 2024-05-28 17:27:48 Cloud cover: 6.674603


Image # 25 2024-05-28 17:27:47 Cloud cover: 6.595111


In [ ]:
#Datos del data frame
df

,Image #,Date,Cloud Cover
0,0,2024-04-03 17:27:46,0.003537
1,1,2024-04-03 17:27:45,0.005912
2,2,2024-04-08 17:27:48,0.000000
3,3,2024-04-08 17:27:47,0.000000
4,4,2024-04-13 17:27:50,60.079706
5,5,2024-04-13 17:27:50,58.581382
6,6,2024-04-18 17:27:49,1.839851
7,7,2024-04-18 17:27:49,2.205424
8,8,2024-04-23 17:27:51,0.001380
9,9,2024-04-23 17:27:50,0.001437


In [ ]:
# create a list of images we want (before, during, after)
landsat_sequence = [0,2,5]

In [ ]:
# Define a region of interest with a buffer zone of 20 km
roi = poi.buffer(20000) # meters

In [ ]:
parameters = {
                'min': 0,
                'max': 3000,
                'dimensions': 800,
                'bands': ['B4', 'B3', 'B2'],
                'region':roi
             }

In [ ]:
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()

    print('Image #',i,date,'Cloud cover:',cloud)

    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

Image # 0 2018-10-07 Cloud cover: 0.05


Image # 2 2018-11-08 Cloud cover: 11.83


Image # 5 2018-12-26 Cloud cover: 5.99


In [ ]:


timestamp_ms = sentinel.first().get('system:time_start').getInfo()
timestamp_s = timestamp_ms / 1000
date_gregorian = datetime.datetime.fromtimestamp(timestamp_s).strftime('%Y-%m-%d %H:%M:%S')

print(date_gregorian)

2024-04-03 17:27:46
